# **CartoCell - inference workflow (Phase 5) - BiaPy's latest version**
___  
  
**CartoCell** is a deep learning-based image processing pipeline for the high-throughput segmentation of whole epithelial cysts acquired at low resolution with minimal human intervention. The official documentation of the workflow is in [BiaPy's CartoCell tutorial](https://biapy.readthedocs.io/en/latest/tutorials/instance_seg/cartocell.html).

<figure>
<center>
<img src='https://raw.githubusercontent.com/BiaPyX/BiaPy-doc/refs/heads/master/source/img/cartocell_pipeline.png' width='800px'/>
<figcaption><b>Figure 1</b>: CartoCell processing phases (from Andrés-San Román et al., 2022).</figcaption></center>
</figure>


**This notebook replicates CartoCell's Phase 5**, i.e., it allows the segmentation of 3D epithelial cysts using a deep learning model trained on a large dataset of low-resolution cysts (see Figure 1, Phase 4, model M2).

___


**CartoCell** relies on the [BiaPy library](https://github.com/BiaPyX/BiaPy), freely available in GitHub: https://github.com/BiaPyX/BiaPy

Please note that **CartoCell** is based on a publication. If you use it successfully for your research please be so kind to cite our work:

*Andres-San Roman, J.A., Gordillo-Vazquez, C., Franco-Barranco, D., Morato, L., Fernandez-Espartero, C.H., Baonza, G., Tagua, A., Vicente-Munuera, P., Palacios, A.M., Gavilán, M.P., Martín-Belmonte, F., Annese, V., Gómez-Gálvez, P., Arganda-Carreras, I., Escudero, L.M. [CartoCell, a high-content pipeline for 3D image analysis, unveils cell morphology patterns in epithelia](https://www.cell.com/cell-reports-methods/fulltext/S2667-2375(23)00249-7). Cell Reports Methods, 3(10).*


___

<font color='red'><b>Note:</b></font> This notebook uses the **latest version of BiaPy** to replicate the pipeline proposed in the original **CartoCell** [manuscript](https://www.cell.com/cell-reports-methods/fulltext/S2667-2375(23)00249-7), thus results may vary slightly from those using the implementation of **BiaPy** at the time of publication.


___


## **Expected inputs and outputs**
___
**Inputs**

This notebook expects two folders as input:
* **Test Raw Images**: Containing the raw 3D images for testing the model. A cyst per image is expected. For optimal performance, the voxel resolution should match that of the training images: 1.62 x 1.62 x 0.5 micron/voxel.
* **[OPTIONAL] Test Label Images**: Containing the corresponding instance-label 3D images of the raw test images. Thus, their quantity and dimensions should match those of the test raw images.
* **Output Folder**: Specify a directory where the segmentation results will be saved.

**Outputs**

If the execution is successful, two folders will be created for each input image containing:
* A **TIFF image** with the cell instances before 3D Voronoi post-processing.
* A **TIFF image** with the cell instances after 3D Voronoi post-processing.


<figure>
<center>
<img src='https://biapy.readthedocs.io/en/latest/_images/cyst_sample.gif' width='300'/>
<img src='https://biapy.readthedocs.io/en/latest/_images/cyst_instance_prediction.gif' width='300'/>
<figcaption><b>Figure 2</b>: Example of input and output images. From left to rigth: 3D TIFF input image and the resulting TIFF image with the cell instances after Voronoi post-processing</figcaption></center>
</figure>


<font color='red'><b>Note</b></font>: for testing purposes, you can also run this notebook with the samples images provided in *Manage file(s) source > Option 3*. Those samples belong to the test set used in our [publication](https://www.cell.com/cell-reports-methods/fulltext/S2667-2375(23)00249-7).

**Data structure**

To ensure the proper operation of the library, the input data directory tree should be something like this:

```
dataset/
└── test
    ├── x
    │   ├── test-0001.tif
    │   ├── test-0002.tif
    │   ├── . . .
    │   └── test-9999.tif
    └── y
        ├── test-0001.tif
        ├── test-0002.tif
        ├── . . .
        └── test-9999.tif
```

**⚠️ Warning:** Ensure that images and their corresponding masks are sorted in the same way. A common approach is to fill with zeros the image number added to the filenames (as in the example).

**Input Format Support**

This notebook is compatible with a range of input formats. You can use the following file extensions: `.tif`, `.npy` (every extension for 3D images supported by [scikit-image](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread)).

## **Prepare the environment**
___

Establish connection with Google services. You **must be logged in to Google** to continue.
Since this is not Google's own code, you will probably see a message warning you of the dangers of running unfamiliar code. This is completely normal.


## **Manage file(s) source**
---
The input folder can be provided using three different options: by directly uploading the folder (option 1), by using a folder stored in your own Google Drive (option 2) or by automatically downloading a few samples of our data (option 3).

Depending on the option chosen, different steps will have to be taken, as explained in the following cells.


### **Option 1: Use your local files and upload them to the notebook**
---
You will be prompted to upload your files to Colab and they will be stored under `/content/input/`.

In [ ]:
#@markdown ##Play the cell to upload local files (test raw images)

from google.colab import files
import os
input_dir = '/content/input/test/x'

if os.path.exists(input_dir):
    # Ask the user if they want to delete the existing items in the folder
    delete_items = ''
    while not delete_items in ['y', 'n']:
        delete_items = input("Do you want to delete the existing items in the folder? (yes[y]/no[n]): ").strip().lower()
    if delete_items == 'y':
        for delete_root, delete_dirs, delete_files in os.walk(input_dir, topdown=False):
            for name in delete_files:
                os.unlink(os.path.join(delete_root, name))
else:
    # Ensure the directory exists
    os.makedirs(input_dir, exist_ok=True)


%cd {input_dir}
uploaded = files.upload()
%cd /content

In [ ]:
#@markdown ## [OPTIONAL] Play the cell to upload local files (test labels)

from google.colab import files
import os
input_dir = '/content/input/test/y'

if os.path.exists(input_dir):
    # Ask the user if they want to delete the existing items in the folder
    delete_items = ''
    while not delete_items in ['y', 'n']:
        delete_items = input("Do you want to delete the existing items in the folder? (yes[y]/no[n]): ").strip().lower()
    if delete_items == 'y':
        for delete_root, delete_dirs, delete_files in os.walk(input_dir, topdown=False):
            for name in delete_files:
                os.unlink(os.path.join(delete_root, name))
else:
    # Ensure the directory exists
    os.makedirs(input_dir, exist_ok=True)


%cd {input_dir}
uploaded = files.upload()
%cd /content

### **Option 2: Mount your Google Drive**
---
To use this notebook on your own data from Google Drive, you need to mount Google Drive first.

Play the cell below to mount your Google Drive and follow the link that will be shown. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

Once this is done, your data will be available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Click on the URL.

#@markdown * Sign in your Google Account.

#@markdown * Copy the authorization code.

#@markdown * Enter the authorization code.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### **Option 3: Download our publication's test samples**
---
Don't you have data readily available but still want to test the notebook? No problem! Simply execute the following cell to download a sample dataset.

Specifically, we'll use the **test set used to evaluate the M2 model** in our manuscript and publicly available in [Zenodo](https://zenodo.org/records/10973241).

In [ ]:
#@markdown ##Play to download the example dataset (publication test set)
!pip install gdown==5.1.0 --quiet
import gdown
import os

os.chdir('/content/')

print( 'Downloading original CartoCell dataset...')

gdown.download("https://drive.google.com/uc?id=1eScAB6mpmPeQcirR5xK6Rkz4AAobsG_X", "CartoCell.zip", quiet=True)

!unzip -q CartoCell.zip
!rm CartoCell.zip

print( 'Dataset downloaded and unzipped under /content/CartoCell')


Dataset downloaded and unzipped under /content/CartoCell



## **Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

Go to **Runtime -> Change the Runtime type**

**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

**Accelerator: GPU** *(Graphics processing unit)*

## **Paths for Input Images and Output Files**
___

Depending on the option you chose for managing file sources, you'll set your paths differently:

- **Option 1 (Upload from Local Machine)**:
  - Set `test_data_path` to `/content/input/test/x`
  - Set `test_data_gt_path` to `/content/input/test/y`
  
- **Option 2 (Use Google Drive Data)**:
  - Insert the paths to your input files and your desired output directory here, i.e., `/content/gdrive/MyDrive/...`.
- **Option 3 (Use Our Sample Data)**:
  - Set `train_data_path` to `/content/CartoCell/test/x`
  - Set `train_data_gt_path` to `/content/CartoCell/test/y`
  - Set `output_path` to `/content/out`

**Note**: Ensure you download your results from the `/content/out` directory after the process!

**Helpful Tip**: If you're unsure about the paths to your folders, look at the top left of this notebook for a small folder icon. Navigate through the directories until you locate your desired folder. Right-click on it and select "Copy Path" to copy the folder's path.

In [ ]:
#@markdown #####Path to test raw images
test_data_path = '/content/CartoCell/test/x' #@param {type:"string"}
#@markdown ##### Select to load ground truth labels (if exist) and measure output performance
load_gt = True #@param {type:"boolean"}
#@markdown #####If you checked the previous option, introduce here the path to the test (ground truth) labels
test_data_gt_path = '/content/CartoCell/test/y' #@param {type:"string"}
#@markdown #####Path to store the resulting images (it'll be created if not existing):
output_path = '/content/output' #@param {type:"string"}

## **Install BiaPy library**
This might take some minutes depending on the current installed libraries in Colab.

In [ ]:
#@markdown ##Play to install BiaPy and its dependences

import os
import sys
import numpy as np
from tqdm.notebook import tqdm
from skimage.io import imread
import ipywidgets as widgets
from ipywidgets import Output

# Uninstall some packages already installed in Colab for Pytorch 2.1.0
!pip uninstall -y torchtext

!pip install biapy==3.5.6

# Then install Pytorch + CUDA 11.8
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118
!pip install 'timm>=0.9.12' torchmetrics[image] pytorch-msssim

from biapy import BiaPy

changed_source = False

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.5/377.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/timm/optim/optim_factory.py:7: FutureWarning: Importing from timm.optim.optim_factory is deprecated, please import via timm.optim
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.optim", FutureWarning)


## **Download pretrained model and apply 3D segmentation workflow (Phase 5)**


In [ ]:
#@markdown ##Play to download and run the pretrained model M2
import os
import errno
import yaml

os.chdir('/content/')

job_name = "cartocell_inference_latest"
checkpoint_file = "/content/model_weights_cartocell.pth"

# remove template file it is exists
template_file = '{}.yaml'.format(job_name)
if os.path.exists( template_file ):
    os.remove( template_file )

# Download .yaml file and model weights
if not os.path.exists("cartocell_inference_latest.yaml"):
    !wget https://raw.githubusercontent.com/BiaPyX/BiaPy/refs/heads/master/templates/instance_segmentation/CartoCell_paper/cartocell_inference_latest.yaml &> /dev/null
    print("\nCartocell yaml configuration file downloaded!")

if not os.path.exists("model_weights_cartocell.pth"):
    #https://drive.google.com/file/d/1CZbakRUOcSTRkob74PnUaNZH_rJ6CNuy/view?usp=share_link
    gdown.download("https://drive.google.com/uc?id=1CZbakRUOcSTRkob74PnUaNZH_rJ6CNuy", "model_weights_cartocell.pth", quiet=True)
    print( '\nM2 model weights successfully downloaded!')

# Edit previous configuration file if it exists to load the checkpoint model
if os.path.exists( template_file ):
    with open( template_file, 'r') as stream:
        try:
            biapy_config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    biapy_config['PATHS'] = {}
    biapy_config['PATHS']['CHECKPOINT_FILE'] = checkpoint_file
    biapy_config['MODEL'] = {}
    biapy_config['MODEL']['LOAD_CHECKPOINT'] = True

    # save file
    with open( template_file, 'w') as outfile:
        yaml.dump(biapy_config, outfile, default_flow_style=False)

# Check folders before modifying the .yaml file
if not os.path.exists(test_data_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), test_data_path)
ids = sorted(next(os.walk(test_data_path))[2])
if len(ids) == 0:
    raise ValueError("No images found in dir {}".format(test_data_path))

if not os.path.exists(template_file):
    raise ValueError("No YAML configuration file found in {}".format(template_file))

if not os.path.exists(checkpoint_file):
    raise ValueError("No h5 checkpoint file found in {}".format(checkpoint_file))


# open template configuration file
with open( template_file, 'r') as stream:
    try:
        biapy_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

biapy_config['DATA']['TEST']['PATH'] = test_data_path
biapy_config['DATA']['TEST']['GT_PATH'] = test_data_gt_path

biapy_config['DATA']['TEST']['LOAD_GT'] = load_gt
biapy_config['TRAIN']['ENABLE'] = False
biapy_config['TEST']['ENABLE'] = True
biapy_config['MODEL']['LOAD_CHECKPOINT']= True
biapy_config['PATHS'] = {}
biapy_config['PATHS']['CHECKPOINT_FILE']= checkpoint_file

# save file
with open( template_file, 'w') as outfile:
    yaml.dump(biapy_config, outfile, default_flow_style=False)

print( "Inference configuration finished.")

job_name = os.path.splitext(template_file)[0].split('/')[-1]

# Run the code
biapy = BiaPy(template_file, result_dir=output_path, name=job_name, run_id=1, gpu=0)
biapy.run_job()




Cartocell yaml configuration file downloaded!

M2 model weights successfully downloaded!
Inference configuration finished.
Date: 2024-12-10 08:54:06
Arguments: Namespace(config='cartocell_inference_latest.yaml', result_dir='/content/output', name='cartocell_inference_latest', run_id=1, gpu=0, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', dist_backend='nccl')
Job: cartocell_inference_latest_1
Python       : 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
PyTorch:  2.4.0+cu118
Not using distributed mode
[08:54:06.536631] Configuration details:
[08:54:06.537964] AUGMENTOR:
  AFFINE_MODE: reflect
  AUG_NUM_SAMPLES: 10
  AUG_SAMPLES: True
  BRIGHTNESS: True
  BRIGHTNESS_FACTOR: (0.1, 0.15)
  BRIGHTNESS_MODE: 3D
  CBLUR_DOWN_RANGE: (2, 8)
  CBLUR_INSIDE: True
  CBLUR_SIZE: (0.2, 0.4)
  CHANNEL_SHUFFLE: False
  CMIX_SIZE: (0.2, 0.4)
  CNOISE_NB_ITERATIONS: (1, 3)
  CNOISE_SCALE: (0.05, 0.1)
  CNOISE_SIZE: (0.2, 0.4)
  CONTRAST: False
  CONTRAST_FACTOR: (-0.1, 0.1)
  C

100%|██████████| 1/1 [00:00<00:00,  2.18it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
100%|██████████| 1/1 [00:00<00:00,  1.91it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                

[08:54:47.129558] Creating X_test channels . . .


  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|       

[08:54:52.003771] DATA.VAL.PATH changed from /path/to/data to /path/to/data_BCM_thick
[08:54:52.003848] DATA.VAL.GT_PATH changed from /path/to/data to /path/to/data_BCM_thick
[08:54:52.003897] DATA.TEST.PATH changed from /content/CartoCell/test/x to /content/CartoCell/test/x_BCM_thick
[08:54:52.003940] DATA.TEST.GT_PATH changed from /content/CartoCell/test/y to /content/CartoCell/test/y_BCM_thick
[08:54:52.004820] ######################
[08:54:52.004884] #   LOAD TEST DATA   #
[08:54:52.004917] ######################
[08:54:52.004982] ### LOAD ###
[08:54:52.005524] ###############
[08:54:52.005589] # Build model #
[08:54:52.005623] ###############
[08:54:52.005741] Extracting model from checkpoint file /content/model_weights_cartocell.pth
[08:54:52.126433] Comparing configurations . . .
[08:54:52.127400] Configurations seem to be compatible. Continuing . . .
[08:55:03.064985] Loading checkpoint from file /content/model_weights_cartocell.pth
[08:55:03.234681] Model weights loaded!
[08:5

...

[09:55:02.894084] Painting FPs . . .



100%|██████████| 11/11 [00:00<00:00, 74.68it/s]


[09:55:03.054541] Saving (1, 128, 127, 152, 3) data as .tif in folder: /content/output/cartocell_inference_latest/results/cartocell_inference_latest_1/instance_associations



100%|██████████| 60/60 [59:59<00:00, 60.00s/it]

[09:55:03.126032] DatasetMatching: {'criterion': 'iou', 'thresh': 0.5, 'fp': 16, 'tp': 37, 'fn': 8, 'precision': 0.6981132075471698, 'recall': 0.8222222222222222, 'accuracy': 0.6065573770491803, 'f1': 0.7551020408163265, 'n_true': 45, 'n_pred': 53, 'mean_true_score': 0.5314031389024523, 'mean_matched_score': 0.6463011148813609, 'panoptic_quality': 0.4880232908287827}
[09:55:03.131388] DatasetMatching: {'criterion': 'iou', 'thresh': 0.75, 'fp': 53, 'tp': 0, 'fn': 45, 'precision': 0, 'recall': 0, 'accuracy': 0, 'f1': 0, 'n_true': 45, 'n_pred': 53, 'mean_true_score': 0.0, 'mean_matched_score': 0.0, 'panoptic_quality': 0.0}
[09:55:03.167182] Releasing memory . . .
[09:55:03.168281] #############
[09:55:03.169029] #  RESULTS  #
[09:55:03.169223] #############
[09:55:03.169254] The values below represent the averages across all test samples
[09:55:03.174208] Test IoU (B channel) (per patch): 0.5404676747825341
[09:55:03.176439] Test IoU (C channel) (per patch): 0.2252431315416593
[09:55:03.1

## **Visualize 3D instance segmentation results**
---

In [ ]:
#@markdown ###Play to visualize results from the test set
#@markdown The current model will be applied to some test images and results will be shown as browsable 2D stacks displaying:
#@markdown 1. The **Source image**.
#@markdown 2. Its corresponding **Ground truth** labels.
#@markdown 3. The model **Prediction** labels using watershed.
#@markdown 4. The model **Post-processed** labels using our Voronoi algorithm.
#@markdown 5. The **Overlay** between ground truth and post-processed labels.

%matplotlib inline
import matplotlib
import numpy as np
from numpy.random import randint, seed
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed
import ipywidgets as widgets
from google.colab import output
output.enable_custom_widget_manager()

final_results = os.path.join(output_path, job_name, 'results', job_name + "_1")

instance_results = os.path.join(final_results, "per_image_instances")
post_results = os.path.join(final_results, "per_image_post_processing")

show_post = os.path.exists( post_results )

# Show a few examples to check that they have been stored correctly
ids_pred = sorted(next(os.walk(instance_results))[2])
ids_input = sorted(next(os.walk(test_data_path))[2])
ids_gt = sorted(next(os.walk(test_data_gt_path))[2])

# create random color map
vals = np.linspace(0,1,256)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.gist_rainbow(vals))
cmap.colors[0] = [0., 0., 0., 1.] # set background to black

samples_to_show = min(len(ids_input), 3)
chosen_images = np.random.choice(len(ids_input), samples_to_show, replace=False)
seed(1)

test_samples = []
test_sample_preds = []
test_sample_posts = []
test_sample_gt = []

# read 3D images again
for i in range(len(chosen_images)):
    aux = imread(os.path.join(test_data_path, ids_input[chosen_images[i]]))
    test_samples.append(aux)

    aux = imread(os.path.join(instance_results, ids_pred[chosen_images[i]])).astype(np.uint16)
    test_sample_preds.append(aux)

    if show_post:
        aux = imread(os.path.join(post_results, ids_pred[chosen_images[i]])).astype(np.uint16)
        test_sample_posts.append(aux)

    aux = imread(os.path.join(test_data_gt_path, ids_gt[chosen_images[i]])).astype(np.uint16)
    test_sample_gt.append(aux)

num_subplots = 5 if show_post else 4

# function to show results in 3D within a widget
def scroll_in_z(z, j):

    plt.figure(figsize=(25,5))
    # Source
    plt.subplot(1,num_subplots,1)
    plt.axis('off')
    plt.imshow(test_samples[j][z-1], cmap='gray')
    plt.title('Source (z = ' + str(z) + ')', fontsize=15)

    # Target (Ground-truth)
    plt.subplot(1,num_subplots,2)
    plt.axis('off')
    plt.imshow(test_sample_gt[j][z-1], cmap=cmap, interpolation='nearest')
    plt.title('Ground truth (z = ' + str(z) + ')', fontsize=15)

    # Prediction
    plt.subplot(1,num_subplots,3)
    plt.axis('off')
    plt.imshow(test_sample_preds[j][z-1], cmap=cmap, interpolation='nearest')
    plt.title('Prediction (z = ' + str(z) + ')', fontsize=15)

    # Post-processed
    if show_post:
        plt.subplot(1,num_subplots,4)
        plt.axis('off')
        plt.imshow(test_sample_posts[j][z-1], cmap=cmap, interpolation='nearest')
        plt.title('Post-processed (z = ' + str(z) + ')', fontsize=15)

    # Overlay
    plt.subplot(1,num_subplots,num_subplots)
    plt.axis('off')
    plt.imshow(test_sample_gt[j][z-1], cmap='Greens')
    plt.imshow(test_sample_posts[j][z-1], alpha=0.5, cmap='Purples')
    plt.title('Overlay (z = ' + str(z) + ')', fontsize=15)

    plt.show()

for j in range(samples_to_show):
    interact(scroll_in_z, z=widgets.IntSlider(min=1, max=test_sample_gt[j].shape[0], step=1, value=test_sample_gt[j].shape[0]//2), j=fixed(j));

interactive(children=(IntSlider(value=62, description='z', max=125, min=1), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=52, description='z', max=105, min=1), Output()), _dom_classes=('widget-i…

interactive(children=(IntSlider(value=66, description='z', max=133, min=1), Output()), _dom_classes=('widget-i…

In [ ]:
#@markdown ###Play to display the path to the output files (one 3D TIFF label image for each input image).

final_results = os.path.join(output_path, job_name, 'results', job_name+"_1")

instance_results = os.path.join(final_results, "per_image_instances")
post_results = os.path.join(final_results, "per_image_post_processing")

print("Output paths:")
print("    Instance segmentation files before post-processing are in {}".format(instance_results))
print("    Instance segmentation files after post-processing are in {}".format(post_results))

[09:58:54.969104] Output paths:
[09:58:54.969252]     Instance segmentation files before post-processing are in /content/output/cartocell_inference_latest/results/cartocell_inference_latest_1/per_image_instances
[09:58:54.969383]     Instance segmentation files after post-processing are in /content/output/cartocell_inference_latest/results/cartocell_inference_latest_1/per_image_post_processing


## **Download segmentation results**
---

In [ ]:
#@markdown ###Play to download a zip file with the segmentation results **before post-processing** in the validation set.

from google.colab import files

!zip -q -j /content/cartocell_val_segmentation_results_before.zip $instance_results/*.tif

files.download("/content/cartocell_val_segmentation_results_before.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@markdown ###Play to download a zip file with all **post-processed** segmentation results in the validation set.

from google.colab import files

!zip -q -j /content/cartocell_val_segmentation_results.zip $post_results/*.tif

files.download("/content/cartocell_val_segmentation_results.zip")


## **Acknowledgments**
---
We would like to express our gratitude for the inspiration drawn from the exceptional [ZeroCostDL4Mic notebooks](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki). Specifically, we've adopted some of their metric and parameter descriptions, as well as the 3D visualization widget code present in their [U-Net 3D notebook](https://colab.research.google.com/github/HenriquesLab/ZeroCostDL4Mic/blob/master/Colab_notebooks/U-Net_3D_ZeroCostDL4Mic.ipynb). Additionally, our heartfelt thanks go out to [Estibaliz Gomez-de-Mariscal](https://scholar.google.es/citations?user=gsg-TAUAAAAJ) for her invaluable support and perceptive feedback, which significantly enhanced the quality of this notebook.
